In [5]:
from arcgis.gis import GIS
from arcgis import features as fs
import glob
import os
import pandas as pd

jgeo_path = '/home/greg/data/rawdata/JornadaGeospatial'

In [6]:
# Some stuff we need to access the NMSU ArcGIS Online system
import sys
sys.path.append('/home/greg/GitHub/jgeo-utils')
sys.path.append('/home/greg/admin/')
import jgeo_py.agol as ag
import nmsu_agol_cred # AGOL credentials
agol_url=nmsu_agol_cred.agol_url
agol_user=nmsu_agol_cred.agol_user
password=nmsu_agol_cred.password
jornadagis_grpid=nmsu_agol_cred.jornadagis_grpid
john_ragosta_grpid = nmsu_agol_cred.john_ragosta_grpid

# Log into NMSU AGOL and get a GIS object
gis = GIS(agol_url, username=agol_user,
    password=password)

# Groups


# Metadata file
metadata = pd.read_csv(os.path.join(jgeo_path, 'metadata', 'JornadaGeosp_metadata_20220107.csv'))

In [7]:
my_content = gis.content.search(query="owner:" + gis.users.me.username, 
                                #item_type="Geopackage", 
                                max_items=5)
my_content

[<Item title:"JRNLandforms2000" type:GeoJson owner:gmaurer1>,
 <Item title:"JERVegetation1915" type:GeoJson owner:gmaurer1>,
 <Item title:"JornadaStudy_004_Lter_weather_station" type:GeoJson owner:gmaurer1>,
 <Item title:"Buffington_Herbel_1915" type:GeoJson owner:gmaurer1>,
 <Item title:"JRNPhysiography2000" type:GeoJson owner:gmaurer1>]

In [5]:
metadata.head()

,package_name,gpkg_fname,layer_name,geom_type,attributes,n_features,origin_gdb,package_abstract,package_tags,layer_abstract,layer_summary,layer_tags,metadata_link,data_link,contact_name,contact_email,license,access_use_text,date_read
0,JornadaVegetation,JornadaVegetation.gpkg,CDRRCGrassCondition1858,MultiPolygon,"['OBJECTID', 'PERIMETER', 'CRGRASS_18', 'CRGRA...",13,All_studies_JornadaGDB_03_30_21.gdb,This layer is part of a collection of Jornada ...,"Jornada Basin, JornadaVegetation, vegetation, ...",layer abstract,layer summary,layer tags,metadata link,data link,Jornada IM Team,jornada.data@nmsu.edu,Creative Commons Attribution 4.0,Restricted to users who have completed the Jor...,2022-01-07 10:31:35.836133
1,JornadaVegetation,JornadaVegetation.gpkg,CDRRCShrubPresence1858,MultiPolygon,"['OBJECTID', 'AREA_', 'PERIMETER', 'CRSHRUB_18...",29,All_studies_JornadaGDB_03_30_21.gdb,This layer is part of a collection of Jornada ...,"Jornada Basin, JornadaVegetation, vegetation, ...",layer abstract,layer summary,layer tags,metadata link,data link,Jornada IM Team,jornada.data@nmsu.edu,Creative Commons Attribution 4.0,Restricted to users who have completed the Jor...,2022-01-07 10:31:35.996706
2,JornadaVegetation,JornadaVegetation.gpkg,JERGrassCondition1858,MultiPolygon,"['OBJECTID', 'PERIMETER', 'JERGRASS_1', 'JERGR...",20,All_studies_JornadaGDB_03_30_21.gdb,This layer is part of a collection of Jornada ...,"Jornada Basin, JornadaVegetation, vegetation, ...",layer abstract,layer summary,layer tags,metadata link,data link,Jornada IM Team,jornada.data@nmsu.edu,Creative Commons Attribution 4.0,Restricted to users who have completed the Jor...,2022-01-07 10:31:36.156854
3,JornadaVegetation,JornadaVegetation.gpkg,JERShrubPresence1858,MultiPolygon,"['OBJECTID', 'GIS_IMS_jer_shrub_presence_18_2'...",32,All_studies_JornadaGDB_03_30_21.gdb,This layer is part of a collection of Jornada ...,"Jornada Basin, JornadaVegetation, vegetation, ...",layer abstract,layer summary,layer tags,metadata link,data link,Jornada IM Team,jornada.data@nmsu.edu,Creative Commons Attribution 4.0,Restricted to users who have completed the Jor...,2022-01-07 10:31:36.314905
4,JornadaVegetation,JornadaVegetation.gpkg,JERVegetation1915,MultiPolygon,"['OBJECTID', 'AREA_', 'PERIMETER', 'SVEG12_', ...",218,All_studies_JornadaGDB_03_30_21.gdb,This layer is part of a collection of Jornada ...,"Jornada Basin, JornadaVegetation, vegetation, ...",layer abstract,layer summary,layer tags,metadata link,data link,Jornada IM Team,jornada.data@nmsu.edu,Creative Commons Attribution 4.0,Restricted to users who have completed the Jor...,2022-01-07 10:31:36.476252


## Copy research project geopackages to AGOL

In [50]:
import importlib as imp
imp.reload(ag)
import pdb

# Get a sorted list of all the project geopackages
prj_gpkgs = sorted(glob.glob(os.path.join(jgeo_path, "jrn_studies", "/*.gpkg")))

# Create a folder for GeoJSON items and for the 
# packages (FeatureLayerCollections)
folder_geojson = 'JornadaResearchProjects_geojson'
gis.content.create_folder(folder_geojson)
folder_pkg = 'JornadaResearchProjects'
gis.content.create_folder(folder_pkg)

for i, pkg_path in enumerate(prj_gpkgs[0:6]):
    pkg_name = pkg_path.split('/')[1][0:6]
    studyid = pkg_name[3:6]
    print(pkg_name)
    # Get the metadata
    pkg_md = metadata[metadata.gpkg_fname==pkg_name+'.gpkg']
    
    # Create a hosted feature service on AGOL
    new_svc = gis.content.create_service(name=pkg_name,
        service_type='featureService', folder=folder_pkg)
    # Create a featureLayerCollection with new_svc
    new_flc = fs.FeatureLayerCollection.fromitem(new_svc)
    # Share the feature service with the JornadaGIS group
    new_svc.share(everyone=False, org=False, groups=[jornadagis_grpid])

    # Update the properties of the new featureLayerCollection (title,
    # summary, description, etc)
    fs_defs = ag.jornada_fs_definitions(new_flc, pkg_md)
    # A little kludge to fix the geopackage description
    fs_defs['description'] = fs_defs['description'].replace(
        ' layer is part of ', ' is ')
    new_flc.manager.update_definition(fs_defs)
    new_svc.update(fs_defs) # This updates the item info?
    # Copy the geopackage layers into the featureLayerCollection
    ag.geopkg_to_multi_fs(gis, folder_geojson, new_flc, pkg_path,
        pkg_md, clean_local=True)

    

prj001
True
True
True
True
True
prj002
True
prj004
True
prj008
True
prj009
True
True
prj010
True
True


In [49]:
folder_geojson = 'JornadaResearchProjects_geojson'
folder_pkg = 'JornadaResearchProjects'
gis.content.delete_folder(folder_geojson)
gis.content.delete_folder(folder_pkg)

Folder doesn't exist.
Folder doesn't exist.


False

## Copy other geopackages

In [7]:
import importlib as imp
imp.reload(ag)

gpkgs = ['JornadaVegetation.gpkg',
    'JornadaBounds&Structures.gpkg',
    'JornadaMonitoringStations.gpkg',
    'JornadaLandforms&Soil.gpkg',
    'JornadaMisc.gpkg']

for i, pkg_path in enumerate(gpkgs[0:5]):
    # Create a folder
    folder_pkg = pkg_path.split('.')[0]
    gis.content.create_folder(folder_pkg)

    # Get the metadata
    pkg_md = metadata[metadata.package_name==folder_pkg]
    
    # Copy the geopackage layers into the featureLayerCollection
    items, fslist = ag.geopkg_to_single_fs(gis, folder_pkg, pkg_path,
        pkg_md, clean_local=True)

    # Share the feature layers with groups
    for i in range(len(fslist)):
        fslist[i].share(everyone=False, org=False,
            groups=[jornadagis_grpid, johntest_grpid])

CDRRCGrassCondition1858
CDRRCShrubPresence1858
JERGrassCondition1858
JERShrubPresence1858
JERVegetation1915
JERVegetation1928
JERVegetation1998
Buffington_Herbel_1858
Buffington_Herbel_1915
Buffington_Herbel_1928
Buffington_Herbel_1963
1938_CDRRC_Vegetation
1998_CDRRC_Vegetation
Boundary_JER_full
Boundary_CDRRC_JER
All3_Bdry_25Aug15
Airstrip
BurnPlots21Aug13
Cattleguards
CDNP_Bdry_25Aug15
CDRRC_Bdry_25Aug15
CDRRC_ExcloEDITED21Aug13
CDRRC_Exclosures_17Sep15
ClosedRoad
CSISParking
DefunctTowers
Dirt_Tanks
ExclosuresJER_17Jul2015
ExclusuresJER_Jul172015
JER_Bdry_25Aug15
JER_CDRRC_Bdry_FromCorrectedMar2014
JER_DeededBndry_OCt2013
JER_FtB_WSMR_bndry
Lvstk_Corral_Lot_Lane
Pastures_CDRRC2_16Sep15
Pastures_JER_16Sep15
RanchStructures
random_troughs
skipping layer: random_troughs
Rds_ClipToJER_27Apr18
RemvdFenceLabel
skipping layer: RemvdFenceLabel
RemvdFences
Tanks
Towers3
TwdieTower_WGS84
UTEP_Site_Access_Trail_UTM
Wells
AllRds_JER_And_Vicinity_04Sep15
Fence_JER_06Nov15
Fence_CDRRC_26Aug15
Jo

In [4]:
folder_geojson = 'JornadaMonitoringStations'
#folder_pkg = 'JornadaResearchProjects'
#gis.content.delete_folder(folder_geojson)
#gis.content.delete_folder(folder_pkg)

TypeError: 'NoneType' object is not subscriptable

In [8]:
items[1]

<Item title:"cr_rg_point" type:GeoJson owner:gmaurer1>

In [9]:
fslist[1]

<Item title:"cr_rg_point" type:Feature Layer Collection owner:gmaurer1>